In [0]:

spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")

import datetime
from applications.db_check_v1.common.base_test import debug
from applications.db_check_v1.cases.store.app_est_publisher_v1.test_app_est_publisher_v1 import \
    TestPublisherEstPreviewDaily, TestPublisherEstFinalWeekly
from applications.db_check_v1.cases.store.app_est_publisher_v1.test_app_est_publisher_aggr_v1 import \
    TestPublisherWeeklyPreAggrEst, TestPublisherMonthlyPreAggrEst, TestPublisherQuarterlyPreAggrEst, \
    TestPublisherYearlyPreAggrEst

t_date = datetime.datetime(2020,10,24)
mycaseList = [
    TestPublisherEstFinalWeekly(trigger_datetime=t_date, methodName='test_publisher_est_final_weekly_etl_completeness'),
    TestPublisherEstFinalWeekly(trigger_datetime=t_date, methodName='test_publisher_est_etl_final_timelines'),
    TestPublisherWeeklyPreAggrEst(trigger_datetime=t_date, methodName='test_publisher_est_pre_aggr_etl_completeness'),
]

debug(mycaseList)

In [0]:


spark.read.format("avro").load("s3://b2c-prod-data-pipeline-raw-rating/raw/rating.v1/insert/date=2022-02-18/market_code=google-play/device_code=android-all/aidp002842f0563f34908e07eee64e1ea189.avro/").show()


In [0]:
%%sh

# aws s3 ls s3://b2c-prod-data-pipeline-qa/aa.store/

ls -al /tmp/pkg_install.log

# aws s3 cp /tmp/pkg_install.log s3://b2c-prod-data-pipeline-qa/zidong/aa.store/pkg_install.log
# aws s3 ls s3://b2c-prod-data-pipeline-qa/
aws s3 ls s3://b2c-prod-data-pipeline-unified-dna/unified/dna.company-hq-mapping.v1/dimension/_update_date=2020-10-10/
aws s3 ls s3://b2c-prod-data-pipeline-unified-dna/unified/dna.company-hq-mapping.v1/dimension/_update_date=2020-10-11/
aws s3 ls s3://b2c-prod-data-pipeline-unified-dna/unified/dna.company-hq-mapping.v1/dimension/_update_date=2020-10-12/
aws s3 ls s3://b2c-prod-data-pipeline-unified-dna/unified/dna.company-hq-mapping.v1/dimension/_update_date=2020-10-13/
aws s3 ls s3://b2c-prod-data-pipeline-unified-dna/unified/dna.company-hq-mapping.v1/dimension/_update_date=2020-10-14/
aws s3 ls s3://b2c-prod-data-pipeline-unified-dna/unified/dna.company-hq-mapping.v1/dimension/_update_date=2020-10-15/

# s3://b2c-prod-data-pipeline-unified-dna/unified/dna.company-hq-mapping.v1/dimension/
# s3://b2c-prod-data-pipeline-unified-dna/unified/dna.app-tag-mapping.v1/dimension/
# s3://b2c-prod-data-pipeline-unified-dna/unified/dna.in-unified-product-product-mapping.v1/dimension/
# s3://b2c-prod-data-pipeline-unified-dna/unified/dna.mapping-log.v1/dimension/


In [0]:


spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import pandas as pd
pd.set_option('expand_frame_repr', False)
import boto3
from datetime import datetime
import pandas as pd
from pyspark.sql import functions as F
from applications.db_check_v1.common.constants import COUNTRY_CODE_MAPPING_BY_MARKET_CODE as COUNTRY_CODE_MAPPING
from conf.settings import *
from applications.db_check_v1.common.db_check_utils import query_df
import pandas as pd

daily_est_dsn =(
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_DAILY_EST_NAME,
        user=PG_DAILY_EST_ACCESS_ID,
        host=PG_DAILY_EST_HOSTS[0][0],
        password=PG_DAILY_EST_SECRET_KEY,
        port=PG_DAILY_EST_HOSTS[0][1]
    )
)


def get_date_list(start_date, end_date, freq="D"):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=start_date, end=end_date, freq=freq))]
    return date_list


def compare(date):
    sql = """
select distinct(app_id) from plproxy.execute_select_nestloop($proxy$ 
    select distinct(app_id)
    from aa.app_store_daily_estimate_{}
    where 
        date = '{}'
$proxy$) tbl (app_id BIGINT);
"""
    print datetime.today()
    db_df = []
    for device in [0,1,2]:
        db_device_df = query_df(daily_est_dsn, sql.format(device, date))
        db_df = db_device_df if len(db_df)==0 else pd.concat([db_df, db_device_df])
    db_df = db_df.drop_duplicates()
    print datetime.today()
    unified_df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/store.app-est.v1/fact/granularity=daily/date={}/".format(date)).select("app_id").distinct().toPandas()
    print datetime.today()
    
    
    diff_df =db_df.merge(unified_df, indicator=True, how='left')
    diff_df = diff_df.loc[diff_df["_merge"] != "both"]
    print diff_df
    
    print datetime.today()
    if len(diff_df) ==  0:
        return "{} PASS".format(date)
    else:
        return "{} FAIL: {} {} {}".format(date, len(unified_df),len(db_df), diff_df.app_id.tolist()[0:5])
        
        
def write_log(strobj, s3obj):
    s3obj.put(Body=str(strobj))


In [0]:

s3 = boto3.resource('s3')
s3object = s3.Object('b2c-prod-data-pipeline-qa', 'tom/all/regression1.txt')
log = [] 

# 2010-09-02/   2020-02-21


date_list = ["2019-09-02"]
for date in date_list:
    temp_log = ""
    temp_log = compare(date)
    print temp_log

    # try:
    #     temp_log = compare(date)
    #     print temp_log
    # except Exception, e:
    #     temp_log = "{}: ERROR".format(date)
    #     print temp_log
    #     print e.message
    # log.append(temp_log)


In [0]:
%md
2019-09-02 FAIL: 901273 900484 [1148468242, 951566153, 438000720, 928368430, 1046307415]


In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/store.app-est.v1/fact/granularity=daily/date=2019-09-02/

In [0]:


spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/store.app-est.v1/fact/granularity=daily/date=2019-09-02/").filter("app_id = 1148468242").show(10)


In [0]:
%%sh


%%sh

PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-4-329644124.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d dailyest -p a7432 << EOF 

select distinct(app_id) from plproxy.execute_select_nestloop(\$proxy\$ 
    select distinct(app_id)
    from aa.app_store_daily_estimate_2
    where 
        date = '2014-01-09'
\$proxy\$) tbl (app_id BIGINT);

EOF



In [0]:
%%sh

date
aws s3 ls s3://b2c-prod-data-pipeline-qa/tom/all2/
echo '******'
aws s3 ls s3://b2c-prod-data-pipeline-qa/tom/gameiq/

In [0]:
%%sh

export D_DATE='2018-12-08'
 echo "*************Legacy ETL Routing $D_DATE *************"
 echo "ios"
aws s3 ls s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/$D_DATE/ios/sbe_est_app/0/ --human | grep "part-"
 echo "gp"
aws s3 ls s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/$D_DATE/android/sbe_est_app/1/ --human | grep "part-"
 echo "*************New ETL Routing $D_DATE *************"
 echo "ios"
aws s3 ls s3://b2c-prod-dca-store-estimates/store_estv2/APP_ESTIMATES_FINAL/version=1.0.0/range_type=DAY/date=$D_DATE/platform=ios/  --human | grep "part-" | tail -n 3
 echo "gp"
aws s3 ls s3://b2c-prod-dca-store-estimates/store_estv2/APP_ESTIMATES_FINAL/version=1.0.0/range_type=DAY/date=$D_DATE/platform=android/  --human | grep "part-" | tail -n 3

#  aws s3 ls s3://b2c-prod-dca-store-estimates/store_estv2/APP_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date=2020-01-31/platform=ios/
#  echo '*************'
#  aws s3 ls s3://b2c-prod-dca-store-estimates/store_estv2/APP_ESTIMATES_INTERMEDIATE/version=2.0.0/range_type=DAY/date=2020-01-31/platform=ios/round=0/
#  echo '*************'
#  aws s3 ls s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2020-01-31/android/sbe_est_app/1/

In [0]:


spark.read.parquet("s3://b2c-prod-dca-store-estimates/store_estv2/APP_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date=2019-09-01/platform=ios/").show(10)
spark.read.parquet("s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2019-09-01/ios/sbe_est_app/0/").show(10)

In [0]:
%%sh

echo "*************Legacy ETL Restatement GP 2020-01-31*************"
aws s3 ls s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2020-01-31/android/sbe_est_app/1/ --human | grep "part-"
echo "*************Legacy ETL Restatement GP 2020-01-17*************"
aws s3 ls s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2020-01-17/android/sbe_est_app/1/ --human | grep "part-"
echo
echo
echo "*************Legacy ETL Routing IOS 2020-01-31*************"
aws s3 ls s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2020-01-31/ios/sbe_est_app/0/ --human | grep "part-"
echo "*************Legacy ETL Routing IOS 2020-01-17*************"
aws s3 ls s3://b2c-prod-dca-store-estimates/store_est/v_final/DAY/2020-01-17/ios/sbe_est_app/0/ --human | grep "part-"


echo "*************UDS ETL Restatement GP 2020-01-31*************"
aws s3 ls s3://b2c-prod-dca-store-estimates/store_estv2/APP_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date=2020-01-31/platform=android/  --human | grep "part-" | tail -n 3
echo "*************UDS ETL Restatement GP 2020-01-17*************"
aws s3 ls s3://b2c-prod-dca-store-estimates/store_estv2/APP_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date=2020-01-17/platform=android/  --human | grep "part-" | tail -n 3
echo
echo
echo "*************UDS ETL Routing IOS 2020-01-31*************"
aws s3 ls s3://b2c-prod-dca-store-estimates/store_estv2/APP_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date=2020-01-31/platform=ios/  --human | grep "part-" | tail -n 3
echo "*************UDS ETL Routing IOS 2020-01-17*************"
aws s3 ls s3://b2c-prod-dca-store-estimates/store_estv2/APP_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date=2020-01-17/platform=ios/  --human | grep "part-" | tail -n 3

In [0]:


import boto3

def read_s3(key):
    print '*'*100 
    print key
    s3 = boto3.resource('s3')
    obj = s3.Object('b2c-prod-data-pipeline-qa', key)
    body = obj.get()['Body'].read()
    print body

read_s3('tom/gameiq/2020-03-13/regression1.txt')
read_s3('tom/gameiq/2020-03-13/regression2.txt')
read_s3('tom/gameiq/2020-03-13/regression3.txt')
read_s3('tom/gameiq/2020-03-13/regression4.txt')
read_s3('tom/gameiq/2020-03-13/regression5.txt')
read_s3('tom/gameiq/2020-03-13/regression1.1.txt')




In [0]:
%%sh

aws s3 ls s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.topic.v1/fact/process_date=2020-06-09


In [0]:
%%sh





